In [0]:
!pip install transformers
!pip install newsapi-python
!pip install flask-ngrok

In [0]:
from flask import Flask
from flask_ngrok import run_with_ngrok
from newsapi import NewsApiClient
import json
import datetime
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
import requests
from bs4 import BeautifulSoup
import torch

In [0]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [0]:
def match_class(target):
    def do_match(tag):
        classes = tag.get('class', [])
        return all(c in classes for c in target)
    return do_match


def get_important_info(articles):
    important = {'articles': []}
    for art in articles['articles']:
        if art['title'] is None or art['description'] is None or \
                art['urlToImage'] is None or art['publishedAt'][:10] is None or \
                art['url'] is None:
            continue
        ness_art = {'title': art['title'],
                    'description': art['description'],
                    'image': art['urlToImage'],
                    'date': art['publishedAt'][:10],
                    'link': art['url']}
        important['articles'].append(ness_art)
    return important


def get_prev_day():
    return str(datetime.date.today()-datetime.timedelta(1))


def generate_text(text, maxlen, model, tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    indexed_tokens = tokenizer.encode(text, add_special_tokens=False, return_tensors='pt')
    indexed_tokens = indexed_tokens.to(device)
    output_sequences = model.generate(
        input_ids=indexed_tokens,
        max_length=maxlen,
        temperature=1,
        top_k=0,
        top_p=0.9,
        repetition_penalty=1.0,
        do_sample=True,
        num_return_sequences=1,
    )
    if len(output_sequences.shape) > 2:
        output_sequences.squeeze_()
    generated_sequences = []
    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        generated_sequence = generated_sequence.tolist()
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        total_sequence = (text + text[len(tokenizer.decode(indexed_tokens[0],
                                                           clean_up_tokenization_spaces=True)):])
        generated_sequences.append(total_sequence)
    return clear(generated_sequences[0])


def clear(string):
    cleared = ''
    for i in range(len(string)-1, 0, -1):
        if string[i:] not in string[:i]:
            cleared = string[:i+1]
            break
    return '.'.join(cleared.split('.')[:-1])+'.'

In [0]:
app = Flask(__name__)
run_with_ngrok(app)
newsapi = NewsApiClient(api_key='ac0d27dde82341969f6645b174c34679')


@app.route('/')
def index():
    return "Hello, World!"


def get_top_articles():
    all_articles = newsapi.get_top_headlines(language='en', page=1, page_size=20)
    important = get_important_info(all_articles)
    return json.dumps(important)


@app.route('/getTags', methods=['GET'])
def get_tags():
    url = "https://yandex.ru/news/export"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html')
    classes = soup.find_all(match_class(['export__item']))
    flag = False
    # anton = [{'title': 'Популярное', 'tag': 'top'}]
    anton = []
    for class_ in classes:
        try:
            tag = class_.contents[0].attrs['href'].split('.rss')[0].split('/')[-1]
            if tag == 'index':
                if not flag:
                    flag = True
                else:
                    break
            anton.append({'title': class_.text, 'tag': tag})
        except:
            break

    return json.dumps(anton, ensure_ascii=False)


@app.route('/getNewsByTag/<string:newsTag>', methods=['GET'])
def get_news_by_tag(newsTag):
    if str(newsTag).lower() == 'top':
        return get_top_articles()
    all_articles = newsapi.get_everything(q=str(newsTag).lower(), language='en', sort_by='relevancy',
                                          page=1, page_size=20, from_param=get_prev_day())
    important = get_important_info(all_articles)
    return json.dumps(important)


@app.route('/genText/<string:prefix>', methods=['GET'])
def gen_text(prefix):
    return json.dumps(generate_text(prefix, np.random.randint(100, 200), model, tokenizer))


if __name__ == '__main__':
    app.run()